In [1]:
import pandas as pd

In [2]:
#import data
df=pd.read_csv('sabio_combine_csv.csv',encoding='utf-8',low_memory=False)

In [6]:
#query kcat
#query interested data
kcat_query=(df['parameter.type']=='kcat')
kcat=df[kcat_query]
print(f'before clean1, the shape is {kcat.shape}')

before clean1, the shape is (27315, 15)


In [35]:
#replace substrate
df_split=kcat['Substrate'].str.split(';',expand=True).fillna('1')
def replace_substrate(row):
    longest=max(row,key=len)
    if len(longest)>=6:
        return longest
    else:
        return None

newSubstrate=df_split.apply(replace_substrate,axis=1)
print(newSubstrate.index==kcat.index)
kcat.loc[:,'Substrate']=newSubstrate


[ True  True  True ...  True  True  True]


Index([     2,      3,      6,      9,     15,     17,     24,     26,     30,
           35,
       ...
       313719, 313722, 313725, 313727, 313731, 313733, 313735, 313740, 314104,
       314156],
      dtype='int64', length=27315)
Index([     2,      3,      6,      9,     15,     17,     24,     26,     30,
           35,
       ...
       313719, 313722, 313725, 313727, 313731, 313733, 313735, 313740, 314104,
       314156],
      dtype='int64', length=27315)


In [108]:
#clean kcat
#clean dataset
##clean1 clean unit UniprotID,startValue
clean_unit=(kcat['parameter.unit']=='s^(-1)')
clean_UniprotID=(kcat['UniprotID'].notna())&(df['UniprotID'].str.len()==6)
clean_startValue=(kcat['parameter.startValue'].notna())
clean_substrate=(kcat['Substrate'].notna())
kcat_cleaned=kcat[clean_unit&clean_UniprotID&clean_startValue&clean_substrate]
#kcat_cleaned_unique=kcat_cleaned.loc[kcat_cleaned.groupby(['Substrate','UniprotID'])['parameter.startValue'].max()]
print(f'after clean1 the shape is {kcat_cleaned_unique.shape}')


after clean1 the shape is (7407, 15)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3260\1346059628.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kcat_cleaned=kcat[clean_unit&clean_UniprotID&clean_startValue&clean_substrate]


In [133]:
data=pd.DataFrame()
data.loc[:,'ECNumber']=kcat_cleaned['ECNumber']
data.loc[:,'Organism']=kcat_cleaned['Organism']
data.loc[:,'Smiles']=''
data.loc[:,'Sequence']=''
data.loc[:,'Substrate']=kcat_cleaned['Substrate']
data.loc[:,'Value']=kcat_cleaned['parameter.startValue']
data.loc[:,'Unit']=kcat_cleaned['parameter.unit']
data.loc[:,'UniprotID']=kcat_cleaned['UniprotID']

In [135]:
data.to_csv('sabio_cleaned.csv',index=False,encoding='utf-8')